In [1]:
from pprint import pprint
from random import uniform
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
from time import sleep
from gestione_input import leggi_float
import locale
import sys

if sys.platform in ['win32', 'cygwin']:
    locale.setlocale(locale.LC_ALL, 'it')
else:
    locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')


    

# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2' 
# ]

fake_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

url_base = 'https://www.subito.it/annunci-italia/vendita/usato/'

search = input('Inserisci le parole chiave che desideri cercare. ')

def scrape_page(page):
    query_str = {
        'q' : search,
        'o' : page
    }


    result = requests.get(url_base, params=query_str, headers=fake_headers)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # controllare se non è una pagina vuota
        no_result = soup.select('div[class*=ErrorLayout_container__]')
        if no_result:
            if page == 1:
                err_msg = f'La ricerca non ha prodotto risultati.'
            else:
                err_msg = f'Si è raggiunta la pagina n. {page} e qui non ci sono più risultati.'
            raise requests.RequestException(err_msg)

        products = soup.select('div.items__item.item-card')
        # pprint(products)


        items= []

        for adv in products:
            data = {
            'nome della nazione' : adv.select('h2[class*=ItemTitle-module_item-title__]'),
            'price' : adv.select('p.price'),
            'town' : adv.select('span[class*=index-module_town__]'),
            'city' : adv.select('span.city'),
            'shipping' : adv.select('span.shipping-badge'), 
            'sold' : adv.select('span.item-sold-badge'),
            'date' : adv.select('span[class*=index-module_date__]'),
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else: 
                    data[key] = None
                
                # oppure si può anche scrivere così
                #data[key] = value[0].get_text() if value else None

        if data['date']:        

            day, time = tuple(data['date'].split('alle'))

            if day.startswith('Oggi'):
                data['date'] = date.today()
            elif day.startswith('Ieri'):
                data['date'] = date.today() + timedelta(days=1)
            else:
                ...



            data.update({
                'price':data['price'].replace('Spedizione disponibile', '')\
                        .replace('\xa0€', '')\
                        # .replace('Urgente', '')\
                        .replace('Venduto', '') ,
                'url': adv.select('a.link')[0]['href'],
            })

            items.append(data)

    return items 
    # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', '')

current_page = 1
all_data = []
max_page = leggi_float('Quante pagine vuoi analizzare?'
                 'Premi invio per tutte le pagine disponibili', None, None)

while True: 
    if not max_page or current_page <= max_page:
        try:
            items = scrape_page(current_page)
            
        except requests.RequestException as err:
            print(err)
            break
        # print(current_page)
        # print(items)

        all_data+= items
        current_page += 1
        sleep(uniform(1, 4)) 

    else:
        break

   
print('N. pagine analizzate:', current_page - 1)
pprint(all_data)

Si è raggiunta la pagina n. 26 e qui non ci sono più risultati.
N. pagine analizzate: 25
[{'city': '(MC)',
  'date': '28 mag alle 11:44',
  'nome della nazione': 'ZX spectrum computer anni 80',
  'price': '500',
  'shipping': None,
  'sold': None,
  'town': 'Porto Recanati ',
  'url': 'https://www.subito.it/videogiochi/zx-spectrum-computer-anni-80-macerata-494757526.htm'},
 {'city': '(RM)',
  'date': '24 mag alle 19:12',
  'nome della nazione': 'Logitech g502 proteus spectrum',
  'price': '20',
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': 'Roma ',
  'url': 'https://www.subito.it/informatica/logitech-g502-proteus-spectrum-roma-494149163.htm'},
 {'city': '(BA)',
  'date': '19 mag alle 21:35',
  'nome della nazione': 'Orologio BIG BOLD Swatch SB01N101',
  'price': '120',
  'shipping': None,
  'sold': None,
  'town': 'Bari ',
  'url': 'https://www.subito.it/abbigliamento-accessori/orologio-big-bold-swatch-sb01n101-bari-493317790.htm'},
 {'city': '(TO)',
  'date': '14 m